# 라이브러리 임포트

In [72]:
import pandas as pd
import requests
import json

REST_API_KEY = '1830579e0e3f456ce3988b002758b9de'

import warnings
warnings.filterwarnings('ignore')

In [73]:
def get_yx(query): # 좌표 받기
    import requests
    headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={query}'
    r = requests.get(url, headers=headers)
    addr = r.json()
    try: # 오류가 날만한 구간
        return [addr['documents'][0]['y'], addr['documents'][0]['x']]
    except: # 오류가 날 경우 반환값
        return '결과없음'
    
def get_address(query): # 구주소 받기
    import requests
    headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={query}'
    r = requests.get(url, headers=headers)
    addr = r.json()
    try:
        return addr['documents'][0]['address']['address_name']
    except:
        return '결과없음'

# 데이터 확인

In [74]:
df = pd.read_excel('전기차_충전소_설치현황_20220921.xls', header=2)
df

,시구,설치장소,주소,급속충전기(대),완속충전기(대),지원차종
0,전체,가평소방서,경기도 가평군 가평읍 대곡리 213-5,1,0,"SM3 Z.E,테슬라"
1,전체,가평종합운동장,경기도 가평군 가평읍 대곡리 316,2,0,"BMW i3,스파크EV,볼트EV"
2,전체,가평지사,경기도 가평군 가평읍 가화로 169-11,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
3,전체,가평하수도사업소,경기도 가평군 가평읍 달전리 364,1,0,"SM3 Z.E,테슬라"
4,전체,상면사무소,경기도 가평군 상면 연하리 171-1,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
...,...,...,...,...,...,...
4676,전체,콘텐츠박물관,경상북도 안동시 동부동 447-8,1,0,"SM3 Z.E,테슬라"
4677,전체,태안화력주차장,충청남도 태안군 원북면 방갈리 831,5,0,"SM3 Z.E,테슬라"
4678,전체,포레나대원아파트,경상남도 창원시 의창구 대원동 40,0,8,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
4679,전체,한국항공우주연구원,대전광역시 유성구 어은동 45,4,0,"SM3 Z.E,테슬라"


# 데이터 전처리

In [75]:
# 필요없는 컬럼 정리
df.drop(columns=['시구', '지원차종'], inplace=True)
df

,설치장소,주소,급속충전기(대),완속충전기(대)
0,가평소방서,경기도 가평군 가평읍 대곡리 213-5,1,0
1,가평종합운동장,경기도 가평군 가평읍 대곡리 316,2,0
2,가평지사,경기도 가평군 가평읍 가화로 169-11,1,1
3,가평하수도사업소,경기도 가평군 가평읍 달전리 364,1,0
4,상면사무소,경기도 가평군 상면 연하리 171-1,1,0
...,...,...,...,...
4676,콘텐츠박물관,경상북도 안동시 동부동 447-8,1,0
4677,태안화력주차장,충청남도 태안군 원북면 방갈리 831,5,0
4678,포레나대원아파트,경상남도 창원시 의창구 대원동 40,0,8
4679,한국항공우주연구원,대전광역시 유성구 어은동 45,4,0


In [76]:
# 새 데이터프레임 생성
df['시'] = df['주소'].str.split().str.get(0)
df_daejeon = df[df['시']=='대전광역시']

In [77]:
# 주소 정제
df_daejeon['주소'] = df_daejeon['주소'].str.split('(').str.get(0).str.split(',').str.get(0).str.strip()
df_daejeon.sample(10)

,설치장소,주소,급속충전기(대),완속충전기(대),시
1928,대자연마을아파트,대전광역시 서구 관저동 990,1,3,대전광역시
3256,드리움2단지 아파트,대전광역시 유성구 대정동 317-1,1,3,대전광역시
3798,센트럴파크3단지아파트,대전광역시 중구 문화동 계백로1716번길 87,1,1,대전광역시
1304,행복한마을 아파트,대전광역시 대덕구 평촌동 540-1,1,2,대전광역시
3277,죽동푸르지오아파트,대전광역시 유성구 죽동 730,1,3,대전광역시
2011,큰마을 아파트,대전광역시 서구 갈마동 1418,2,0,대전광역시
1946,무궁화아파트,대전광역시 서구 월평동 223,1,0,대전광역시
1937,도안13블럭 금성백조 예미지 아파트,대전광역시 서구 도안동 965,1,0,대전광역시
1980,전원 아파트,대전광역시 서구 월평동 312,0,2,대전광역시
1955,삼성래미안 아파트1단지,대전광역시 서구 가장동 55,1,3,대전광역시


In [78]:
#인덱스 리셋
df_daejeon.reset_index(inplace=True)
df_daejeon

,index,설치장소,주소,급속충전기(대),완속충전기(대),시
0,1298,대전대덕구청 주차장,대전광역시 대덕구 대전로 1033번길 20,2,0,대전광역시
1,1299,법동생활체육공영주차장,대전광역시 대덕구 법동 439-1,1,0,대전광역시
2,1300,송촌공영주차장,대전광역시 대덕구 송촌동 478-2,1,0,대전광역시
3,1301,송촌주민센터 공영주차장,대전광역시 대덕구 송촌동 460-2,1,0,대전광역시
4,1302,한국수자원공사,대전광역시 대덕구 신탄진로 200,2,0,대전광역시
...,...,...,...,...,...,...
136,3830,태평전통시장,대전광역시 중구 태평동 333-8,2,0,대전광역시
137,3831,하우스토리2차아파트,대전광역시 중구 문화동 계룡로 922,0,2,대전광역시
138,4665,대전복용승마장,대전광역시 유성구 덕명동 산 10-2,2,0,대전광역시
139,4675,카이스트,대전광역시 유성구 어은동 44,2,0,대전광역시


In [79]:
df_daejeon['주소'] = df_daejeon['주소'].apply(get_address)
df_daejeon['위도'] = df_daejeon['주소'].apply(get_yx).str.get(0)
df_daejeon['경도'] = df_daejeon['주소'].apply(get_yx).str.get(1)

In [9]:
df_daejeon.sample(10) # 주소, 위,경도에 결과없음. 과감히 삭제하자.

,index,설치장소,주소,급속충전기(대),완속충전기(대),시,위도,경도
82,3250,대전지방기상청,대전 유성구 구성동 20-3,1,0,대전광역시,36.3722555191821,127.371637367991
133,3805,쌍용예가아파트,대전 중구 태평동 554,1,2,대전광역시,36.3234713809484,127.396972272104
9,1359,대전충남직할,대전 동구 용전동 9-7,4,13,대전광역시,36.3585891623784,127.435332487035
126,3785,문창시장 제2공영주차장,대전 중구 문창동 122-8,1,0,대전광역시,36.3154981452706,127.436413645182
23,1920,구봉마을9단지아파트,대전 서구 관저동 1154,1,4,대전광역시,36.2948151843972,127.331863713755
107,3275,전력연구원,대전 유성구 문지동 103-16,2,0,대전광역시,36.3911933886835,127.391016724066
75,3243,노은해랑숲마을5단지,대전 유성구 지족동 1033,1,3,대전광역시,36.3852606479143,127.304740478006
29,1931,대전광역시청 부설주차장,대전 서구 둔산동 1420,2,0,대전광역시,36.3501712979595,127.384838607761
103,3271,이지아파트,대전 유성구 상대동 438-16,0,1,대전광역시,36.3476696276071,127.332540708268
63,3231,구암평생학습센터,대전 유성구 구암동 621-7,1,0,대전광역시,36.3492324477718,127.330298814487


In [10]:
df_daejeon.info() # 위,경도 실수타입으로 변경해야 됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     141 non-null    int64 
 1   설치장소      141 non-null    object
 2   주소        141 non-null    object
 3   급속충전기(대)  141 non-null    int64 
 4   완속충전기(대)  141 non-null    int64 
 5   시         141 non-null    object
 6   위도        141 non-null    object
 7   경도        141 non-null    object
dtypes: int64(3), object(5)
memory usage: 8.9+ KB


In [80]:
for i in df_daejeon.index:
    if df_daejeon['주소'][i] == '결과없음':
        df_daejeon = df_daejeon.drop([i])

In [12]:
df_daejeon.sample(10)

,index,설치장소,주소,급속충전기(대),완속충전기(대),시,위도,경도
47,1962,상아 아파트,대전 서구 만년동 1,1,0,대전광역시,36.3697832662245,127.37812082365
113,3281,한국가스기술공사 본사,대전 유성구 봉산동 839,2,0,대전광역시,36.4360572662149,127.390051451155
102,3270,원신흥도서관,대전 유성구 원신흥동 560,1,0,대전광역시,36.3376688105074,127.33755520989
62,3230,가나파로스빌1차 아파트,대전 유성구 지족동 898-1,0,2,대전광역시,36.3756046948669,127.31685620348
110,3278,카이스트 문지캠퍼스,대전 유성구 문지동 103-6,1,0,대전광역시,36.3930531303574,127.399512821914
124,3775,대사동 제7노외공영주차장,대전 중구 대사동 206-3,2,0,대전광역시,36.3140952157846,127.422152586918
95,3263,신안인스빌리베라,대전 유성구 원신흥동 488,1,3,대전광역시,36.3428756223028,127.345537377057
69,3237,노은도서관,대전 유성구 지족동 871-6,1,0,대전광역시,36.3815387879291,127.321223187946
105,3273,장대푸르지오아파트,대전 유성구 장대동 479,1,2,대전광역시,36.3613513039044,127.332142532983
78,3246,대전 현충원,대전 유성구 갑동 394-34,5,0,대전광역시,36.3605747850454,127.290267897755


In [81]:
# df_daejeon['위도'] = df_daejeon['위도'].astype('float')
# df_daejeon['경도'] = df_daejeon['경도'].astype('float')

#df_daejeon = df_daejeon.astype({'급속충전기(대)':'float', '완속충전기(대)':'float', '위도':'float', '경도':'float'})
df_daejeon[['급속충전기(대)', '완속충전기(대)', '위도', '경도']] = df_daejeon[['급속충전기(대)', '완속충전기(대)', '위도', '경도']].astype('float')
df_daejeon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 140
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     138 non-null    int64  
 1   설치장소      138 non-null    object 
 2   주소        138 non-null    object 
 3   급속충전기(대)  138 non-null    float64
 4   완속충전기(대)  138 non-null    float64
 5   시         138 non-null    object 
 6   위도        138 non-null    float64
 7   경도        138 non-null    float64
dtypes: float64(4), int64(1), object(3)
memory usage: 13.8+ KB


In [70]:
df_daejeon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 140
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     138 non-null    int64  
 1   설치장소      138 non-null    object 
 2   주소        138 non-null    object 
 3   급속충전기(대)  138 non-null    float64
 4   완속충전기(대)  138 non-null    float64
 5   시         138 non-null    object 
 6   위도        138 non-null    float64
 7   경도        138 non-null    float64
dtypes: float64(4), int64(1), object(3)
memory usage: 13.8+ KB


# 지역별 인구 데이터

In [139]:
data = pd.read_excel('202208_202208_주민등록인구및세대현황_월간.xlsx', header=2).drop(columns=['행정기관코드', '남여 비율'])
data.head()

,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수
0,서울특별시,"9,488,454","4,472,975",2.12,"4,601,295","4,887,159"
1,서울특별시 종로구,"143,499","73,866",1.94,"69,408","74,091"
2,서울특별시 종로구 청운효자동,"11,766","5,198",2.26,"5,392","6,374"
3,서울특별시 종로구 사직동,"9,278","4,668",1.99,"4,124","5,154"
4,서울특별시 종로구 삼청동,"2,384","1,188",2.01,"1,130","1,254"


In [140]:
data = data.rename(columns={'세대당 인구':'세대당인구', '남자 인구수':'남자', '여자 인구수':'여자'})
data.head()

,행정기관,총인구수,세대수,세대당인구,남자,여자
0,서울특별시,"9,488,454","4,472,975",2.12,"4,601,295","4,887,159"
1,서울특별시 종로구,"143,499","73,866",1.94,"69,408","74,091"
2,서울특별시 종로구 청운효자동,"11,766","5,198",2.26,"5,392","6,374"
3,서울특별시 종로구 사직동,"9,278","4,668",1.99,"4,124","5,154"
4,서울특별시 종로구 삼청동,"2,384","1,188",2.01,"1,130","1,254"


In [141]:
data['시'] = data['행정기관'].str.split().str.get(0)
data

,행정기관,총인구수,세대수,세대당인구,남자,여자,시
0,서울특별시,"9,488,454","4,472,975",2.12,"4,601,295","4,887,159",서울특별시
1,서울특별시 종로구,"143,499","73,866",1.94,"69,408","74,091",서울특별시
2,서울특별시 종로구 청운효자동,"11,766","5,198",2.26,"5,392","6,374",서울특별시
3,서울특별시 종로구 사직동,"9,278","4,668",1.99,"4,124","5,154",서울특별시
4,서울특별시 종로구 삼청동,"2,384","1,188",2.01,"1,130","1,254",서울특별시
...,...,...,...,...,...,...,...
3860,제주특별자치도 서귀포시 서홍동,"11,253","4,629",2.43,"5,551","5,702",제주특별자치도
3861,제주특별자치도 서귀포시 대륜동,"15,529","6,966",2.23,"7,854","7,675",제주특별자치도
3862,제주특별자치도 서귀포시 대천동,"13,837","6,477",2.14,"7,011","6,826",제주특별자치도
3863,제주특별자치도 서귀포시 중문동,"12,244","5,939",2.06,"6,200","6,044",제주특별자치도


In [142]:
data_daejeon = data[data['시'] == '대전광역시']
data_daejeon.head()

,행정기관,총인구수,세대수,세대당인구,남자,여자,시
1103,대전광역시,"1,448,240","672,491",2.15,"722,486","725,754",대전광역시
1104,대전광역시 동구,"220,724","108,585",2.03,"111,184","109,540",대전광역시
1105,대전광역시 동구 중앙동,"4,284","2,971",1.44,"2,415","1,869",대전광역시
1106,대전광역시 동구 효동,"24,195","9,545",2.53,"11,852","12,343",대전광역시
1107,대전광역시 동구 신인동,"14,926","6,797",2.20,"7,358","7,568",대전광역시


In [143]:
# data_daejeon['총인구수'] = data_daejeon['총인구수'].str.replace(',', '').astype('int')
# data_daejeon['세대수'] = data_daejeon['세대수'].str.replace(',', '').astype('int')
# data_daejeon['남자'] = data_daejeon['남자'].str.replace(',', '').astype('int')
# data_daejeon['여자'] = data_daejeon['여자'].str.replace(',', '').astype('int')
data_daejeon[['총인구수', '세대수', '남자', '여자']].replace('[^\w]', '', regex=True, inplace=True)

In [144]:
data_daejeon.head()

,행정기관,총인구수,세대수,세대당인구,남자,여자,시
1103,대전광역시,"1,448,240","672,491",2.15,"722,486","725,754",대전광역시
1104,대전광역시 동구,"220,724","108,585",2.03,"111,184","109,540",대전광역시
1105,대전광역시 동구 중앙동,"4,284","2,971",1.44,"2,415","1,869",대전광역시
1106,대전광역시 동구 효동,"24,195","9,545",2.53,"11,852","12,343",대전광역시
1107,대전광역시 동구 신인동,"14,926","6,797",2.20,"7,358","7,568",대전광역시


In [21]:
data_daejeon['시'] = data_daejeon['행정기관'].str.split().str.get(0)
data_daejeon['구'] = data_daejeon['행정기관'].str.split().str.get(1)
data_daejeon['동'] = data_daejeon['행정기관'].str.split().str.get(2)

In [22]:
data_daejeon

,행정기관,총인구수,세대수,세대당인구,남자,여자,시,구,동
1103,대전광역시,1448240,672491,2.15,722486,725754,대전광역시,NaN,NaN
1104,대전광역시 동구,220724,108585,2.03,111184,109540,대전광역시,동구,NaN
1105,대전광역시 동구 중앙동,4284,2971,1.44,2415,1869,대전광역시,동구,중앙동
1106,대전광역시 동구 효동,24195,9545,2.53,11852,12343,대전광역시,동구,효동
1107,대전광역시 동구 신인동,14926,6797,2.20,7358,7568,대전광역시,동구,신인동
...,...,...,...,...,...,...,...,...,...
1186,대전광역시 대덕구 석봉동,16211,6509,2.49,8293,7918,대전광역시,대덕구,석봉동
1187,대전광역시 대덕구 덕암동,13456,6631,2.03,7177,6279,대전광역시,대덕구,덕암동
1188,대전광역시 대덕구 목상동,6383,3223,1.98,3487,2896,대전광역시,대덕구,목상동
1189,대전광역시 대덕구 법1동,12902,5991,2.15,6074,6828,대전광역시,대덕구,법1동


In [23]:
data_daejeon.dropna(inplace=True)
data_daejeon

,행정기관,총인구수,세대수,세대당인구,남자,여자,시,구,동
1105,대전광역시 동구 중앙동,4284,2971,1.44,2415,1869,대전광역시,동구,중앙동
1106,대전광역시 동구 효동,24195,9545,2.53,11852,12343,대전광역시,동구,효동
1107,대전광역시 동구 신인동,14926,6797,2.20,7358,7568,대전광역시,동구,신인동
1108,대전광역시 동구 판암1동,12123,5877,2.06,5910,6213,대전광역시,동구,판암1동
1109,대전광역시 동구 판암2동,8277,5207,1.59,4023,4254,대전광역시,동구,판암2동
...,...,...,...,...,...,...,...,...,...
1186,대전광역시 대덕구 석봉동,16211,6509,2.49,8293,7918,대전광역시,대덕구,석봉동
1187,대전광역시 대덕구 덕암동,13456,6631,2.03,7177,6279,대전광역시,대덕구,덕암동
1188,대전광역시 대덕구 목상동,6383,3223,1.98,3487,2896,대전광역시,대덕구,목상동
1189,대전광역시 대덕구 법1동,12902,5991,2.15,6074,6828,대전광역시,대덕구,법1동


In [24]:
data_daejeon.reset_index().drop(columns=['index'])

,행정기관,총인구수,세대수,세대당인구,남자,여자,시,구,동
0,대전광역시 동구 중앙동,4284,2971,1.44,2415,1869,대전광역시,동구,중앙동
1,대전광역시 동구 효동,24195,9545,2.53,11852,12343,대전광역시,동구,효동
2,대전광역시 동구 신인동,14926,6797,2.20,7358,7568,대전광역시,동구,신인동
3,대전광역시 동구 판암1동,12123,5877,2.06,5910,6213,대전광역시,동구,판암1동
4,대전광역시 동구 판암2동,8277,5207,1.59,4023,4254,대전광역시,동구,판암2동
...,...,...,...,...,...,...,...,...,...
77,대전광역시 대덕구 석봉동,16211,6509,2.49,8293,7918,대전광역시,대덕구,석봉동
78,대전광역시 대덕구 덕암동,13456,6631,2.03,7177,6279,대전광역시,대덕구,덕암동
79,대전광역시 대덕구 목상동,6383,3223,1.98,3487,2896,대전광역시,대덕구,목상동
80,대전광역시 대덕구 법1동,12902,5991,2.15,6074,6828,대전광역시,대덕구,법1동


In [25]:
data_daejeon.구.value_counts()

서구     24
중구     17
동구     16
유성구    13
대덕구    12
Name: 구, dtype: int64

In [26]:
# ;; 도안동 행정동 목록에 왜 없어 ?
for i in data_daejeon.index:
    if data_daejeon['동'][i] == '도안동':
        data_daejeon = data_daejeon.drop([i])

# 차트 나타내기

In [27]:
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# data_daejeon = data_daejeon.groupby('구')['총인구수'].sum().sort_values(ascending=False).head(5)
# data_daejeon = data_daejeon.reset_index()

In [29]:
# sns.barplot(data=data_daejeon,x='구',y='총인구수')

# 지도 시각화 (컬러플레이스)

In [30]:
import json
import folium

In [31]:
jsonfile = open('HangJeongDong_ver20220401.geojson', 'r', encoding='utf-8').read()
jsondata = json.loads(jsonfile)

jsondata_daejeon = {'type':'FeatureCollection'}
jsondata_pick = []
jsondata_dong = [] # 어느 동이 선택되었는지 확인하기 위함

for item in jsondata['features']: # features 리스트 값의 딕셔너리들을 하나 하나 가져와서 그 안의 'sidonm'을 찾는다.
    # print(item['properties']['sidonm'])
    if item['properties']['sidonm'] == '대전광역시':
        dong = item['properties']['adm_nm'].split()[-1].strip()
        item['id'] = dong 
        jsondata_dong.append(dong)
        jsondata_pick.append(item)
        
jsondata_daejeon['features'] = jsondata_pick

In [32]:
json_dong = sorted(jsondata_dong)
json_dong

['가수원동',
 '가양1동',
 '가양2동',
 '가장동',
 '갈마1동',
 '갈마2동',
 '관저1동',
 '관저2동',
 '관평동',
 '괴정동',
 '구즉동',
 '기성동',
 '내동',
 '노은1동',
 '노은2동',
 '노은3동',
 '대동',
 '대사동',
 '대청동',
 '대화동',
 '대흥동',
 '덕암동',
 '도마1동',
 '도마2동',
 '둔산1동',
 '둔산2동',
 '둔산3동',
 '만년동',
 '목동',
 '목상동',
 '문창동',
 '문화1동',
 '문화2동',
 '법1동',
 '법2동',
 '변동',
 '복수동',
 '부사동',
 '비래동',
 '산내동',
 '산성동',
 '삼성동',
 '상대동',
 '석교동',
 '석봉동',
 '성남동',
 '송촌동',
 '신성동',
 '신인동',
 '신탄진동',
 '오류동',
 '오정동',
 '온천1동',
 '온천2동',
 '용두동',
 '용문동',
 '용운동',
 '용전동',
 '원신흥동',
 '월평1동',
 '월평2동',
 '월평3동',
 '유천1동',
 '유천2동',
 '은행선화동',
 '자양동',
 '전민동',
 '정림동',
 '중리동',
 '중앙동',
 '중촌동',
 '진잠동',
 '탄방동',
 '태평1동',
 '태평2동',
 '판암1동',
 '판암2동',
 '학하동',
 '홍도동',
 '회덕동',
 '효동']

In [33]:
data_dong = sorted(data_daejeon['동']) #도안동이 인구데이터에는 있는데 행정동 목록에는 없다 ? ;;
data_dong

['가수원동',
 '가양1동',
 '가양2동',
 '가장동',
 '갈마1동',
 '갈마2동',
 '관저1동',
 '관저2동',
 '관평동',
 '괴정동',
 '구즉동',
 '기성동',
 '내동',
 '노은1동',
 '노은2동',
 '노은3동',
 '대동',
 '대사동',
 '대청동',
 '대화동',
 '대흥동',
 '덕암동',
 '도마1동',
 '도마2동',
 '둔산1동',
 '둔산2동',
 '둔산3동',
 '만년동',
 '목동',
 '목상동',
 '문창동',
 '문화1동',
 '문화2동',
 '법1동',
 '법2동',
 '변동',
 '복수동',
 '부사동',
 '비래동',
 '산내동',
 '산성동',
 '삼성동',
 '상대동',
 '석교동',
 '석봉동',
 '성남동',
 '송촌동',
 '신성동',
 '신인동',
 '신탄진동',
 '오류동',
 '오정동',
 '온천1동',
 '온천2동',
 '용두동',
 '용문동',
 '용운동',
 '용전동',
 '원신흥동',
 '월평1동',
 '월평2동',
 '월평3동',
 '유천1동',
 '유천2동',
 '은행선화동',
 '자양동',
 '전민동',
 '정림동',
 '중리동',
 '중앙동',
 '중촌동',
 '진잠동',
 '탄방동',
 '태평1동',
 '태평2동',
 '판암1동',
 '판암2동',
 '학하동',
 '홍도동',
 '회덕동',
 '효동']

In [34]:
len(json_dong), len(data_dong) 

(81, 81)

In [35]:
for i, item in enumerate(zip(data_dong, json_dong)):
    print(i,item)

0 ('가수원동', '가수원동')
1 ('가양1동', '가양1동')
2 ('가양2동', '가양2동')
3 ('가장동', '가장동')
4 ('갈마1동', '갈마1동')
5 ('갈마2동', '갈마2동')
6 ('관저1동', '관저1동')
7 ('관저2동', '관저2동')
8 ('관평동', '관평동')
9 ('괴정동', '괴정동')
10 ('구즉동', '구즉동')
11 ('기성동', '기성동')
12 ('내동', '내동')
13 ('노은1동', '노은1동')
14 ('노은2동', '노은2동')
15 ('노은3동', '노은3동')
16 ('대동', '대동')
17 ('대사동', '대사동')
18 ('대청동', '대청동')
19 ('대화동', '대화동')
20 ('대흥동', '대흥동')
21 ('덕암동', '덕암동')
22 ('도마1동', '도마1동')
23 ('도마2동', '도마2동')
24 ('둔산1동', '둔산1동')
25 ('둔산2동', '둔산2동')
26 ('둔산3동', '둔산3동')
27 ('만년동', '만년동')
28 ('목동', '목동')
29 ('목상동', '목상동')
30 ('문창동', '문창동')
31 ('문화1동', '문화1동')
32 ('문화2동', '문화2동')
33 ('법1동', '법1동')
34 ('법2동', '법2동')
35 ('변동', '변동')
36 ('복수동', '복수동')
37 ('부사동', '부사동')
38 ('비래동', '비래동')
39 ('산내동', '산내동')
40 ('산성동', '산성동')
41 ('삼성동', '삼성동')
42 ('상대동', '상대동')
43 ('석교동', '석교동')
44 ('석봉동', '석봉동')
45 ('성남동', '성남동')
46 ('송촌동', '송촌동')
47 ('신성동', '신성동')
48 ('신인동', '신인동')
49 ('신탄진동', '신탄진동')
50 ('오류동', '오류동')
51 ('오정동', '오정동')
52 ('온천1동', '온천1동')
53 ('온천2동', '온천2동')
54 ('용

In [36]:
df_daejeon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 140
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     138 non-null    int64  
 1   설치장소      138 non-null    object 
 2   주소        138 non-null    object 
 3   급속충전기(대)  138 non-null    int64  
 4   완속충전기(대)  138 non-null    int64  
 5   시         138 non-null    object 
 6   위도        138 non-null    float64
 7   경도        138 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 13.8+ KB


In [37]:
daejeon_loc = [df_daejeon['위도'].mean(), df_daejeon['경도'].mean()]

In [38]:
m = folium.Map(location=daejeon_loc, zoom_start=11) # 평균좌표를 중심좌표로 설정

folium.Choropleth(geo_data = jsondata_daejeon,
                  data = data_daejeon, 
                  columns = ['동', '총인구수'],
                  fill_color='OrRd',
                  key_on = 'feature.id').add_to(m)


m

# 충전소 위치 마크 찍기

In [39]:
df_daejeon = df_daejeon.drop(columns=['index'])

In [40]:
df_daejeon

,설치장소,주소,급속충전기(대),완속충전기(대),시,위도,경도
0,대전대덕구청 주차장,대전 대덕구 오정동 500,2,0,대전광역시,36.346594,127.415696
1,법동생활체육공영주차장,대전 대덕구 법동 439-1,1,0,대전광역시,36.375687,127.428060
2,송촌공영주차장,대전 대덕구 송촌동 478-2,1,0,대전광역시,36.363540,127.440423
3,송촌주민센터 공영주차장,대전 대덕구 송촌동 460-2,1,0,대전광역시,36.366417,127.438950
4,한국수자원공사,대전 대덕구 연축동 산 6-2,2,0,대전광역시,36.396937,127.426831
...,...,...,...,...,...,...,...
136,태평전통시장,대전 중구 태평동 333-8,2,0,대전광역시,36.325877,127.397916
137,하우스토리2차아파트,대전 중구 문화동 1-31,0,2,대전광역시,36.319982,127.414607
138,대전복용승마장,대전 유성구 덕명동 산 10-2,2,0,대전광역시,36.352452,127.304864
139,카이스트,대전 유성구 어은동 44,2,0,대전광역시,36.373404,127.358847


In [41]:
df_daejeon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 140
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   설치장소      138 non-null    object 
 1   주소        138 non-null    object 
 2   급속충전기(대)  138 non-null    int64  
 3   완속충전기(대)  138 non-null    int64  
 4   시         138 non-null    object 
 5   위도        138 non-null    float64
 6   경도        138 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 12.7+ KB


In [42]:
df_daejeon = df_daejeon.astype({'급속충전기(대)':'float', '완속충전기(대)':'float'})
df_daejeon.head()

,설치장소,주소,급속충전기(대),완속충전기(대),시,위도,경도
0,대전대덕구청 주차장,대전 대덕구 오정동 500,2.0,0.0,대전광역시,36.346594,127.415696
1,법동생활체육공영주차장,대전 대덕구 법동 439-1,1.0,0.0,대전광역시,36.375687,127.428060
2,송촌공영주차장,대전 대덕구 송촌동 478-2,1.0,0.0,대전광역시,36.363540,127.440423
3,송촌주민센터 공영주차장,대전 대덕구 송촌동 460-2,1.0,0.0,대전광역시,36.366417,127.438950
4,한국수자원공사,대전 대덕구 연축동 산 6-2,2.0,0.0,대전광역시,36.396937,127.426831


In [43]:
df_daejeon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 140
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   설치장소      138 non-null    object 
 1   주소        138 non-null    object 
 2   급속충전기(대)  138 non-null    float64
 3   완속충전기(대)  138 non-null    float64
 4   시         138 non-null    object 
 5   위도        138 non-null    float64
 6   경도        138 non-null    float64
dtypes: float64(4), object(3)
memory usage: 12.7+ KB


In [44]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(m)

In [45]:
# # 일반 마커
# for n in df_daejeon.index:
#   folium.Marker([df_daejeon.loc[n, '위도'], df_daejeon.loc[n, '경도']], tooltip=df_daejeon['설치장소'][n],
#                 icon=folium.Icon(icon='fa-bolt', prefix='fa', color='green')).add_to(m)

In [46]:
# # 써클 마커
# for n in df_daejeon.index:
#     cnt = df_daejeon.loc[n, '급속충전기(대)'] + df_daejeon.loc[n, '완속충전기(대)']
#     folium.CircleMarker([df_daejeon['위도'][n],df_daejeon['경도'][n]], radius=cnt*5, tooltip=df_daejeon['설치장소'][n], 
#                         color='green', fill_color='green').add_to(m)
# # reset_index 하면서 인덱스 컬럼(object)이 생겼고 그게 반복문에 들어가면서 문제가 됐다.

In [51]:
for i in df_daejeon.index:
    folium.Marker([df_daejeon['위도'][i], df_daejeon['경도'][i]],
                  tooltip=df_daejeon['설치장소'][i]).add_to(marker_cluster)

In [ ]:
# for i in range(len(df_daejeon)):
#     if df_daejeon['주소'][i] != '결과없음':
#         folium.Marker([df_daejeon.loc[i, '위도'], df_daejeon.loc[i, '경도']],
#                       tooltip=df_daejeon.loc[i, '설치장소']).add_to(marker_cluster)

In [52]:
m

In [53]:
df_daejeon.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            131, 132, 133, 134, 135, 136, 137, 138, 139, 140],
           dtype='int64', length=138)

In [54]:
range(len(df_daejeon))

range(0, 138)